<a href="https://colab.research.google.com/github/mohsenh17/jaxLearning/blob/main/flax/basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install -U flax treescope

In [ ]:
from flax import nnx
import jax
from jax import numpy as jnp


# Simple Linear

In [ ]:
class Linear(nnx.Module):

  def __init__(self, din:int, dout:int, *, rngs:nnx.Rngs): # * force the succeeding variables must be provided with keyword arg
    key = rngs.params()
    self.w = nnx.Param(jax.random.uniform(key, (din, dout)))
    self.b = nnx.Param(jnp.zeros((dout,)))
    self.din, self.dout = din, dout

  def __call__(self, x:jnp.ndarray):
    return jnp.dot(x, self.w) + self.b

model = Linear(3, 5, rngs=nnx.Rngs(params=0))
x = jnp.ones((6, 3))
y = model(x)
print(y)
nnx.display(model)


[[1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]]
Linear(
  w=Param(
    value=Array(shape=(3, 5), dtype=float32)
  ),
  b=Param(
    value=Array(shape=(5,), dtype=float32)
  ),
  din=3,
  dout=5
)


# Stateful computation
How to update states during forward pass (batchNorm etc)

In [ ]:
class Count(nnx.Variable): pass

class Counter(nnx.Module):
  def __init__(self):
    self.count = Count(jnp.array(0))

  def __call__(self):
    self.count.value += 1

counter = Counter()
print(f'{counter.count.value = }')
counter()
print(f'{counter.count.value = }')

counter.count.value = Array(0, dtype=int32, weak_type=True)
counter.count.value = Array(1, dtype=int32, weak_type=True)


# Nested Modules

In [21]:
class MLP(nnx.Module):
  def __init__(self, din:int, dmid:int, dout:int, *, rngs:nnx.Rngs):
    self.lin1 = Linear(din, dmid, rngs=rngs)
    self.dropout = nnx.Dropout(rate=0.5, rngs=rngs)
    self.bn = nnx.BatchNorm(dmid, rngs=rngs)
    self.lin2 = Linear(dmid, dout, rngs=rngs)
  def __call__(self, x:jnp.ndarray):
    x = self.lin1(x)
    x = self.bn(x)
    x = self.dropout(x)
    x = nnx.gelu(x)
    x = self.lin2(x)
    return x

model = MLP(2, 16, 5, rngs=nnx.Rngs(0))

y = model(x=jnp.ones((3, 2)))

nnx.display(model)
print(y)

MLP(
  lin1=Linear(
    w=Param(
      value=Array(shape=(2, 16), dtype=float32)
    ),
    b=Param(
      value=Array(shape=(16,), dtype=float32)
    ),
    din=2,
    dout=16
  ),
  dropout=Dropout(rate=0.5, broadcast_dims=(), deterministic=False, rng_collection='dropout', rngs=Rngs(
    default=RngStream(
      key=RngKey(
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      ),
      count=RngCount(
        value=Array(5, dtype=uint32),
        tag='default'
      )
    )
  )),
  bn=BatchNorm(
    mean=BatchStat(
      value=Array(shape=(16,), dtype=float32)
    ),
    var=BatchStat(
      value=Array(shape=(16,), dtype=float32)
    ),
    scale=Param(
      value=Array(shape=(16,), dtype=float32)
    ),
    bias=Param(
      value=Array(shape=(16,), dtype=float32)
    ),
    num_features=16,
    use_running_average=False,
    axis=-1,
    momentum=0.99,
    epsilon=1e-05,
    dtype=None,
    param_dtype=<class 'jax.numpy.float32'>,
    use_